In [34]:
import csv
import math
import sympy as sy

In [71]:
class Curve:
    def __init__(self,x=[],y=[]):
        if len(x)==len(y):
            self.x,self.y = (x,y)
        else:
            raise Exception(input("x and y donot have the same length"))
    def read_from_csv(self, filename):
        with open(filename,'r') as csv_file:
            reader = csv.reader(csv_file)
            next(reader, None)  # Skip the header.
            # Unpack the row directly in the head of the for loop.
            for x,y in reader:
                # Convert the numbers to floats.
                self.x.append(float(x))
                self.y.append(float(y))
    def __repr__(self):
        return 'curve'+str(set(zip(self.x,self.y)))
    def __str__(self):
        return str(set(zip(self.x,self.y)))
    def __call__(self,x):
        new_list = [t-x for t in self.x]
        for i in range(0,len(new_list)-1):
            if new_list[i]<0:
                new_list[i]=-new_list[i] 
        new_dict = dict(zip(new_list,self.y))
        return new_dict[sorted(new_dict.keys())[0]]
class FwdYldCurve(Curve):
    def __init__(self,x=[],y=[]):
        Curve.__init__(self,x,y)
    def read_from_csv(self,filename):
        Curve.read_from_csv(self,filename)
class ParYldCurve(Curve):
    def __init__(self,x=[],y=[]):
        Curve.__init__(self,x,y)
    def read_from_csv(self,filename):
        Curve.read_from_csv(self,filename)
    def generateOfForward(self):
        t = [self.x[0]]
        r = [self.y[0]]
        cpnPV = ParYldCurve.fwdNPV(r[0],t[0])
        dfi = sy.exp(-r[0]*t[0])
        df = [dfi]
        for i in range(1,len(self.x)-4):
            t.append(self.x[i])
            dt=t[i]-t[i-1]
            yi=self.y[i]
            c=yi*cpnPV-1
            var('x')
            fwd = nsolve(dfi*(yi/x+(1-yi/x)*sy.exp(-x*dt))+c, x, (yi,))
            dfi*=sy.exp(-fwd*dt)
            cpnPV += ParYldCurve.fwdNPV(fwd,dt)*dfi
            r.append(fwd)
            df.append(dfi)
        self.fwdCrv=FwdYldCurve(t,r)
        return self.fwdCrv
    def fwdNPV(r,dt):
        return (1-sy.exp(-r*dt))/r
if __name__ == "__main__": 
    ParYldCurve().read_from_csv('TreasuryYieldCrv20200212.csv')
    list_maturities = [0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.25, 1.5]
    result_list = []
    for item in list_maturities:
        result_list.append(ParYldCurve().__call__(item))
    with open('result.out', 'w') as f: #Writing output file 
        writer = csv.writer(f)
        writer.writerows([result_list])
    with open('20200212TreasuryFwdCrv.csv', 'w') as f: #Writing output file 
        writer = csv.writer(f)
        writer.writerows(eval(str(ParYldCurve().generateOfForward())))